# 전환이 일어나지 않은 유저(only view)에게 view_ratio 기준으로 추천

# 라이브러리 로드

In [1]:
!pip install implicit
!pip install fastparquet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
import numpy as np
import scipy.sparse as sparse
import random
import implicit
import datetime
from tqdm import tqdm

# 데이터 로드

In [3]:
try:
  path = 'C:/Users/User/Desktop/AIB_13/CP2/data/'
  df = pd.read_parquet(path + 'light_2019-Oct.parquet', engine='fastparquet')
except:
  path = '/content/drive/MyDrive/CP2/data/'
  df = pd.read_parquet(path + 'light_2019-Oct.parquet', engine='fastparquet')

#add_term 데이터 불러오기
try:
  path = 'C:/Users/User/Desktop/AIB_13/CP2/data/'
  add_term = pd.read_parquet(path + 'add_term.parquet', engine='fastparquet')
except:
  path = '/content/drive/MyDrive/CP2/data/'
  add_term = pd.read_parquet(path + 'add_term.parquet', engine='fastparquet')

# 11월 넘어가는 유저 제거

In [5]:
df['event_time'] = df['event_time'].apply(lambda x : x[:-4]).astype('datetime64')
df['event_time'].dtype

dtype('<M8[ns]')

In [6]:
df['event_time'] = df['event_time'] + datetime.timedelta(hours=4)
df = df.loc[df['event_time'] < '2019-11-01']

In [7]:
df.tail()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
42398925,2019-10-31 23:59:59,view,12718908,452985123,NaN,yokohama,96.269997,536689953,0a77064e-0bc6-4745-b703-5e8208bf8f6e
42398926,2019-10-31 23:59:59,view,28719204,-352320551,apparel.shoes,alpina,220.080002,534245704,699164ab-a8f6-4ce0-bd07-b059ff3e1db2
42398927,2019-10-31 23:59:59,view,1005217,-1769995873,electronics.smartphone,xiaomi,151.070007,560095437,15e037ed-4f14-44ba-b7c0-0c033a6c3649
42398928,2019-10-31 23:59:59,view,1003892,-1769995873,electronics.smartphone,xiaomi,144.729996,566243306,566c6d0a-2cd5-4248-ac7b-3af675c05fd3
42398929,2019-10-31 23:59:59,view,17100010,947913283,NaN,NaN,77.199997,512824822,dee78bdc-718a-42bd-8b78-a26259cd1078


# 데이터 전처리

In [8]:
data = df[['user_id','product_id','event_type']]

## view만 존재하는 유저 데이터 추출

1. cart, purchase 이력이 있는 유저들의 id값 받아오기
2. 위에서 얻은 cart, purchase 이력이 있는 유저들을 제외하기

In [9]:
drop_user_id = data.loc[data['event_type'] != 'view', 'user_id']

In [10]:
data = data.loc[~data['user_id'].isin(drop_user_id)].reset_index(drop=True)
data['event_type'].unique()

['view']
Categories (3, object): ['cart', 'purchase', 'view']

In [11]:
# event_type dtype 을 object로 변경

data['event_type'] = data['event_type'].astype('object')

## view 비율 구하기
- 해당 제품을 본 수 / 해당 유저가 전체 제품을 본 수

In [12]:
data.head()

,user_id,product_id,event_type
0,554748717,3900821,view
1,519107250,17200506,view
2,550050854,1307067,view
3,535871217,1004237,view
4,555447699,17300353,view


In [13]:
grouped = data.groupby(['user_id','product_id'])['event_type'].count()
grouped = grouped.reset_index()
grouped =grouped.rename(columns = {'event_type' : 'view_count'})

In [14]:
grouped

,user_id,product_id,view_count
0,33869381,7002639,1
1,64078358,10600284,1
2,183503497,22200103,1
3,184265397,6902133,2
4,184265397,6902303,2
...,...,...,...
15346912,566243205,1004739,1
15346913,566243206,1004741,1
15346914,566243215,1005156,1
15346915,566243216,1005156,1


In [15]:
total_event_type = data.groupby(['user_id'])['product_id'].count()
total_event_type = total_event_type.reset_index()
total_event_type = total_event_type.rename(columns={'product_id' : 'total_view'})

In [16]:
total_event_type

,user_id,total_view
0,33869381,1
1,64078358,1
2,183503497,1
3,184265397,6
4,195082191,1
...,...,...
2538077,566243205,1
2538078,566243206,1
2538079,566243215,1
2538080,566243216,1


In [17]:
data = grouped.merge(total_event_type, on='user_id')

In [18]:
data['view_ratio'] =  (data['view_count'] / data['total_view']) * 100
data.head()

,user_id,product_id,view_count,total_view,view_ratio
0,33869381,7002639,1,1,100.000000
1,64078358,10600284,1,1,100.000000
2,183503497,22200103,1,1,100.000000
3,184265397,6902133,2,6,33.333333
4,184265397,6902303,2,6,33.333333


## add_term 데이터와 합치기

In [19]:
temp = data.merge(add_term, how='left', left_on=['user_id','product_id'], right_on=['user_id','product_id'])

In [20]:
temp = temp.drop_duplicates(['user_id','product_id'])

In [21]:
temp['term'].describe()

count                       296199
mean     0 days 00:57:42.289715195
std      0 days 11:10:14.500085298
min                0 days 00:00:00
25%                0 days 00:00:18
50%                0 days 00:00:40
75%                0 days 00:01:24
max               28 days 00:18:32
Name: term, dtype: object

term이 0days가 나온 행의 view_count를 1로 맞추어줍니다.
이 유저들은 오류로 인하여 중복 처리된것으로 판단합니다.

In [22]:
temp.loc[temp['term'] == temp['term'].min(), 'view_count'] = 1

In [23]:
temp.loc[temp['term'] == temp['term'].min(), 'term'] = temp['term'].median()
temp.loc[temp['term'].isnull(), 'term'] = temp['term'].median()

In [24]:
scores = []
for i in tqdm(temp['term'].dt.seconds):
  if i < 40:
    scores.append(3)
  elif 40 <= i < 60:
    scores.append(2)
  elif 60 <= i :
    scores.append(1)

100%|██████████| 15346917/15346917 [00:06<00:00, 2525059.91it/s]


In [25]:
data['score']= scores

In [26]:
data['weight'] = data['view_ratio'] * data['score']

In [27]:
data.head()

,user_id,product_id,view_count,total_view,view_ratio,score,weight
0,33869381,7002639,1,1,100.000000,2,200.000000
1,64078358,10600284,1,1,100.000000,2,200.000000
2,183503497,22200103,1,1,100.000000,2,200.000000
3,184265397,6902133,2,6,33.333333,2,66.666667
4,184265397,6902303,2,6,33.333333,3,100.000000


In [28]:
data = data[['user_id','product_id','weight']]

# Item Table(product lookup) 테이블 만들기

In [29]:
product_lookup = df[['product_id','category_code','brand']].drop_duplicates('product_id').reset_index(drop=True).sort_values('product_id')
product_lookup.head()

,product_id,category_code,brand
151915,1000978,electronics.smartphone,NaN
8437,1001588,electronics.smartphone,meizu
85152,1001606,electronics.smartphone,apple
32556,1002042,electronics.smartphone,samsung
9400,1002062,electronics.smartphone,samsung


## Rating Matrix 만들기

In [30]:
num_user = data['user_id'].nunique()
num_item = data['product_id'].nunique()

num_user, num_item

(2538082, 159250)

In [31]:
users = list(np.sort(data['user_id'].unique()))
products = list(data['product_id'].unique())
weight = list(data['weight'])

rows = data['user_id'].astype('category').cat.codes
data['user_id_code'] = data['user_id'].astype('category').cat.codes

cols = data['product_id'].astype('category').cat.codes
data['product_id_code'] = data['product_id'].astype('category').cat.codes

len(users), len(products), len(weight)

(2538082, 159250, 15346917)

In [32]:
user_item_matrix = sparse.csr_matrix((weight, (rows, cols)), shape=(num_user, num_item))
user_item_matrix

<2538082x159250 sparse matrix of type '<class 'numpy.float64'>'
	with 15346917 stored elements in Compressed Sparse Row format>

# 추천 시스템 class 구현

In [33]:
from implicit.als import AlternatingLeastSquares
from sklearn import metrics
class MyALS():
  """
  implicit 라이브러리를 이용하여 필요한 기능을 담았습니다.

  Parameters
  ----------
  model : implicit 라이브러리의 AlternatingLeastSquares() 클래스로 생성된 인스턴스
  user_item_matrix : 사용자가 정의한 User-Item Matrix(Sparse Matrix)
  item_lookup : item(product)에 대한 정보를 담은 테이블
  data : User-Item Matrix를 만든 원본데이터

  """
  def __init__(self, model, user_item_matrix, item_lookup, data):

    self.model = model
    self.metrics_model = model
    self.user_item_matrix = user_item_matrix
    self.item_lookup = item_lookup
    self.data = data

  def fit(self, user_item_matrix):
    """
    행렬분해의 ALS를 이용하여 모델을 학습합니다.

    Parameters
    ----------
    user_item_matrix : 사용자가 정의한 User-Item Matrix(Sparse Matrix)
    """
    self.model.fit(user_item_matrix)
    
  def _user_id_2_code(self, user_id):
    """
    입력받은 user_id를 User_Item_Matrix에 있는 user_id_code로 바꾸어주는 함수

    Parameters
    ----------
    user_id : 유저 ID

    """
    user_id_code = self.data.loc[self.data['user_id'] == user_id, 'user_id_code'].unique()[0]
    return user_id_code

  def _product_id_2_code(self, product_id):
    """
    입력받은 product_id를 User-Item-Matrix에 있는 product_id_code로 바꾸어주는 함수

    Parameters
    ----------
    product_id : 상품 ID
    """
    product_id_code = self.data.loc[self.data['product_id'] == product_id, 'product_id_code'].unique()[0]
    return product_id_code

  def _code_2_product_id(self, product_id_code):
    """
    입력받은 product_id_code를 User-Item-Matrix에 있는 product_id로 바꾸어주는 함수

    Parameters
    ----------
    product_id_code : 상품 ID code    
    """
    product_id = self.data.loc[self.data['product_id_code'] == product_id_code, 'product_id'].unique()[0]
    return product_id

  def get_recom_product(self, user_id, n = 10):
    """  
    user_id에 맞는 product를 n개 만큼 추천하여 데이터프레임 형태로 반환하는 함수

    Parameters
    ----------
    user_id : 유저 ID
    n : 추천 받게 될 item의 수
    """ 

    # user_id_2_code 함수를 이용하여 유저의 ID를 user_id_code로 변환합니다
    user_id_code = self._user_id_2_code(user_id)
  
    # model의 recommend를 이용하여 추천받는 제품의 id를 추출합니다.
    # 이때 추천 받는 제품의 id는 product_id가 아니라 product_id_code 입니다.
    recommended = self.model.recommend(user_id_code, self.user_item_matrix[user_id_code], N=n)[0]
    #결과를 담을 리스트를 초기화 합니다.
    results = []
    # 추천 받은 id를 돌면서 item_lookup 테이블에서 해당 product의 정보를 찾아 결과에 담습니다.
    for product_id_code in recommended:
      
      recommended_product_id = self._code_2_product_id(product_id_code)
      result = self.item_lookup.loc[self.item_lookup['product_id'] == recommended_product_id]
      results.append(result)
      
    return pd.concat(results)
  
  def get_user_topN_product(self,user_id,column, n = 20):
    """
    유저가 특정 기준값이 높은 제품 N개를 반환

    Parameters
    ----------
    user_id : 유저 ID
    column : 값을 확인할 기준이 되는 컬럼
    n : 반환할 Item 수
    """
    #입력받은 user_id 를 기준으로 column이 높은 순으로 정렬하여 product_id를 추출
    product_ids = self.data.loc[self.data['user_id'] == user_id].sort_values(column, ascending=False)[:n]['product_id'].values
    #입력받은 user_id 를 기준으로 column이 높은 순으로 정렬하여 column을 추출
    product_values = self.data[self.data['user_id'] == user_id].sort_values(column, ascending=False)[:n][column].values

    results = []
    #item_lookup 테이블에서 id에 맞는 데이터프레임을 찾음
    for i in product_ids:
      result = self.item_lookup.loc[self.item_lookup['product_id'] == i]
      results.append(result)

    #결과를 확인하기 쉽게 데이터프레임으로 반환
    frame = pd.concat(results)
    frame[column] = product_values

    return frame

  def get_explain(self, user_id, item_id, column):
    """
    사용자에게 제품이 추천된 이유를 반환하는 함수

    Parameters
    ----------
    user_id : 유저 ID
    item_id : Item(product) ID
    column : 확인할 컬럼
    """
    #입력받은 user_id, item_id를 user_id_code, product_id_code로 바꾸어줌
    user_id_code = self._user_id_2_code(user_id)
    product_id_code = self._product_id_2_code(item_id)

    #implicit라이브러리의 explain 함수를 사용하여 결과값을 반환
    total_score, top_contributions, user_weights = self.model.to_cpu().explain(user_id_code, self.user_item_matrix, product_id_code)

    results = []
    categorys = []
    brands = []
    scores = []
    # id에 해당하는 user_id, product_id, column, category, brand를 찾기
    for id_, score_ in top_contributions:
      product_id = self._code_2_product_id(id_)
      result = self.data.loc[(self.data['product_id'] == product_id) & (self.data['user_id'] == user_id)][['user_id','product_id',column]]
      category = self.item_lookup.loc[self.item_lookup['product_id'] == product_id, 'category_code'].unique()[0]
      brand = self.item_lookup.loc[self.item_lookup['product_id'] == product_id, 'brand'].unique()[0]

      results.append(result)
      categorys.append(category)
      brands.append(brand)
      scores.append(score_)

    #결과를 확인하기 쉽게 데이터프레임으로 반환
    frame = pd.concat(results)
    frame['score'] = scores
    frame['category'] = categorys
    frame['brand'] = brands
    
    frame = frame[['user_id', 'product_id','category','brand',column, 'score']]
    return frame, total_score

  def _get_train_test(self,percentage=.2, seed=42):
    """
    score를 구하기 위하여 train, test 데이터를 나누어주는 함수
    파라미터로 들어오는 percentage만큼 train_set의 값을 0으로 만들어 줌
    test_set는 기존의 User-Item Matrix에서 0이 아닌값으로 모두 1로 만들어 줌

    Parameters
    ----------
    percentage : 감추고 싶은 데이터의 비율
    seed : random seed

    """
    #원본 데이터를 test_set, train_set에 복사
    test_set = self.user_item_matrix.copy()
    train_set = self.user_item_matrix.copy()

    #relevant(선호 혹은 평가)여부를 확인하기 위하여 test_set에서 0이 아닌값을 1로 만들어 줌 
    test_set[test_set != 0] = 1

    #train_set에서 0이 아닌 x축, y축을 추출
    nonzero_idxs = train_set.nonzero()
    #x, y를 짝을지어 저장
    nonzero_pairs = list(zip(nonzero_idxs[0], nonzero_idxs[1]))

    #랜덤 시드를 적용
    random.seed(seed)
    #주어진 비율로 샘플을 추출
    n_samples = int(np.ceil(percentage * len(nonzero_pairs)))
    samples = random.sample(nonzero_pairs, n_samples)

    
    user_idxs = [index[0] for index in samples]
    item_idxs = [index[1] for index in samples]

    #샘플에 해당 하는 값들을 평가한적이 없도록 보이기 위하여 0으로 감춤
    train_set[user_idxs, item_idxs] = 0
    
    train_set.eliminate_zeros()

    self.zero_user_idxs = user_idxs
    self.zero_item_idxs = item_idxs
    self.train_set = train_set
    self.test_set = test_set
    

  def get_score(self, percentage =.2, seed=42, k = 10, method='hit_at_k', n_samples = 10000):
    """
    train_set로 학습하고 test_set와 비교하여 method 파라미터를 이용하여 추천시스템의 성능을 평가하는 함수
    
    Parameters
    ----------
    percentage : 감추고 싶은 데이터의 비율, default = .2
    seed : random seed, default = 42
    k : 추천할 아이템의 수, default = 10
    method : 평가 지표 - hit_at_k, precision_at_k
    n_samples : 평가할 user의 수, default = 10000

    """
    # 입력받은 n_samples가 최대값이상 이면 최대값으로 적용
    max_num = self.data['user_id_code'].nunique()
    if max_num <= n_samples:
      n_samples = max_num

    # train, test로 데이터가 분리
    self._get_train_test(percentage, seed)
    # metrics_model이 학습
    self.metrics_model.fit(self.train_set)
    # 샘플 user를 랜덤으로 추출
    random_state = np.random.RandomState(seed)
    user_id_code_samples = random_state.choice(self.data['user_id_code'], n_samples)

    #method 방법에 따라 scores 값을 반환
    if method == 'hit_at_k':
      scores = self._hit_at_k(user_id_code_samples, k)
    elif method =='presicion_at_k':
      scores = self._precision_at_k(user_id_code_samples, k)
    return scores

  def _hit_at_k(self, user_id_code_samples, k):
    """
    k개의 추천 중 relevant한것(존재)이 있다면 1, 아니면 0을 반환하여 측정
    추천을 받은 user 수 만큼 나누어 평균을 반환

    Parameters
    ----------
    user_id_code_samples : 샘플링한 유저의 ID 리스트
    k : 추천할 아이템의 수
    """
    
    scores = []
    #입력받은 user_id_code_samples를 돌면서
    for user_id_code_sample in tqdm(user_id_code_samples):
      #해당 유저에게 추천하는 아이템을 추출
      recommedation_ids = self.metrics_model.recommend(user_id_code_sample, self.train_set[user_id_code_sample], N=k)[0]

      results = []
      # 추천 받은 아이템이 유저가 선호(혹은 평가)했는지 확인
      for id_ in recommedation_ids:
        result = self.test_set[user_id_code_sample, id_]
        results.append(result)
      # 만약 결과 리스트안에 1이 있다면 1을 입력, 아니면 0을 입력
      if 1 in results:
        scores.append(1)
      else:
        scores.append(0)
        #결과를 평균내어 반환
    return np.mean(scores)

  def _precision_at_k(self, user_id_code_samples, k):
    """
    k개의 추천 중 사용자가 relevant(선호 혹으 평가)한 아이템이 얼마나 존재하는지 측정
    추천을 받은 user 수 만큼 나누어 평균을 반환

    Parameters
    ----------
    user_id_code_samples : 샘플링한 유저의 ID 리스트
    k : 추천할 아이템의 수
    """
    scores = []
    #입력받은 user_id_code_samples를 돌면서
    for user_id_code_sample in tqdm(user_id_code_samples):
      #해당 유저에게 추천하는 아이템을 추출
      recommedation_ids = self.metrics_model.recommend(user_id_code_sample, self.train_set[user_id_code_sample], N=k)[0]
      results = []
       # 추천 받은 아이템이 유저가 선호(혹은 평가)했는지 확인
      for id_ in recommedation_ids:
        result = self.test_set[user_id_code_sample, id_]
        results.append(result)
      # 유저가 추천받은 아이템들을 얼마나 선호(혹은 평가)했는지 추출
      scores.append(np.mean(results))
      # 결과를 평균내어 반환
    return np.mean(scores)

# 추천하기

In [34]:
model = MyALS(AlternatingLeastSquares(), user_item_matrix, product_lookup, data)

In [35]:
model.fit(user_item_matrix)

  0%|          | 0/15 [00:00<?, ?it/s]

다음은 512475445 유저에 관하여 확인해보겠습니다.

해당 유저는 우리 사이트에서 제품을 본 수가 가장 많은 유저입니다.

In [36]:
user = 512475445
column = 'weight'

model.get_user_topN_product(user, column)

,product_id,category_code,brand,weight
868,4700419,auto.accessories.videoregister,sho-me,6.132329
763,4700478,auto.accessories.videoregister,sho-me,6.051641
4726,4700557,auto.accessories.videoregister,sho-me,5.675094
4864,4700590,auto.accessories.videoregister,sho-me,5.540613
14595,5800802,electronics.audio.subwoofer,kenwood,5.298548
2019,5700788,auto.accessories.player,kenwood,5.190963
2814,5701062,auto.accessories.player,pioneer,4.948897
4495,5701086,auto.accessories.player,pioneer,4.948897
487,5700384,auto.accessories.player,pioneer,3.415815
23871,6100194,auto.accessories.radar,NaN,3.308230


In [37]:
model.get_recom_product(user)

,product_id,category_code,brand
1786,5701128,auto.accessories.player,NaN
753,5701166,auto.accessories.player,NaN
2267,5801218,electronics.audio.subwoofer,NaN
4729,5701246,auto.accessories.player,NaN
367,5700518,auto.accessories.player,NaN
2345,5701247,auto.accessories.player,NaN
690,4700630,auto.accessories.videoregister,NaN
5682,5700282,auto.accessories.player,alpine
8364,5700850,auto.accessories.player,alpine
7023,5701020,auto.accessories.player,NaN


In [38]:
explain_frame, total_score = model.get_explain(user, 5701128, column)
explain_frame

/usr/local/lib/python3.7/dist-packages/implicit/utils.py:29: UserWarning: OpenBLAS detected. Its highly recommend to set the environment variable 'export OPENBLAS_NUM_THREADS=1' to disable its internal multithreading
  "OpenBLAS detected. Its highly recommend to set the environment variable "


,user_id,product_id,category,brand,weight,score
243725,512475445,5701086,auto.accessories.player,pioneer,4.948897,0.136596
243703,512475445,5700788,auto.accessories.player,kenwood,5.190963,0.091284
243693,512475445,5700384,auto.accessories.player,pioneer,3.415815,0.082891
243682,512475445,4700478,auto.accessories.videoregister,sho-me,6.051641,0.052711
243718,512475445,5701062,auto.accessories.player,pioneer,4.948897,0.052308
243763,512475445,5801482,electronics.audio.subwoofer,pioneer,3.093061,0.049371
243722,512475445,5701079,auto.accessories.player,jvc,2.689618,0.045453
243764,512475445,5801483,electronics.audio.subwoofer,pioneer,3.227542,0.041458
243719,512475445,5701063,auto.accessories.player,pioneer,3.039268,0.040616
243704,512475445,5700791,auto.accessories.player,sony,3.308230,0.033771


In [39]:
user = 519607186
column = 'weight'

model.get_user_topN_product(user, column)

,product_id,category_code,brand,weight
5969,1004887,electronics.smartphone,oppo,147.126437
1166,1005159,electronics.smartphone,xiaomi,2.298851
50,1002544,electronics.smartphone,apple,2.298851
171,15100367,NaN,NaN,2.298851
4187,14701391,furniture.living_room.cabinet,NaN,2.298851
1259,14701558,furniture.living_room.cabinet,brw,2.298851
19848,14701705,furniture.living_room.cabinet,brw,1.532567
40247,14700394,furniture.living_room.cabinet,NaN,1.532567
52673,14700738,furniture.living_room.cabinet,brw,1.532567
2286,15100147,NaN,lider,1.532567


In [40]:
model.get_recom_product(user)

,product_id,category_code,brand
141,1004838,electronics.smartphone,oppo
1113,1004886,electronics.smartphone,oppo
585,1004839,electronics.smartphone,oppo
1055,1004961,electronics.smartphone,oppo
2392,1004990,electronics.smartphone,oppo
120205,1005205,electronics.smartphone,NaN
524,1005021,electronics.smartphone,oppo
460,1004902,electronics.smartphone,oppo
6382,1004578,electronics.smartphone,oppo
141303,1005238,electronics.smartphone,NaN


In [41]:
explain_frame, total_score = model.get_explain(user, 1004838, column)
explain_frame

,user_id,product_id,category,brand,weight,score
5080310,519607186,1004887,electronics.smartphone,oppo,147.126437,1.015660
5080302,519607186,1002544,electronics.smartphone,apple,2.298851,0.038671
5080317,519607186,1005159,electronics.smartphone,xiaomi,2.298851,0.027790
5080309,519607186,1004856,electronics.smartphone,samsung,0.766284,0.023213
5080308,519607186,1004794,electronics.smartphone,xiaomi,0.766284,0.009299
5080315,519607186,1005115,electronics.smartphone,apple,0.766284,0.009037
5080311,519607186,1005008,electronics.smartphone,xiaomi,0.766284,0.007011
5080306,519607186,1004433,electronics.smartphone,samsung,0.766284,0.006398
5080320,519607186,1005239,electronics.smartphone,NaN,0.766284,0.005322
5080313,519607186,1005062,electronics.smartphone,xiaomi,0.766284,0.003939


# 성능 평가하기

In [42]:
scores = model.get_score(method='hit_at_k')
print()
print('hit@k(10) : ',round(scores, 2))

  0%|          | 0/15 [00:00<?, ?it/s]

100%|██████████| 10000/10000 [00:12<00:00, 795.89it/s]


hit@k(10) :  0.36


In [43]:
scores = model.get_score(method='presicion_at_k')
print()
print('precision@k(10) : ',round(scores, 2))

  0%|          | 0/15 [00:00<?, ?it/s]

100%|██████████| 10000/10000 [00:12<00:00, 772.02it/s]


precision@k(10) :  0.05
